In [2]:
import pymongo
from collections import Counter

In [3]:
c = pymongo.MongoClient()

In [4]:
db0 = c.neuro1337
db1 = c.neuro1338
db2 = c.neuro1339

In [72]:
class CoolDict(dict):
    def __getattr__(self, name):
        attr = self[name]
        if type(attr) == dict:
            return CoolDict(attr)
        return attr


class Block(CoolDict):
    def __init__(self, arg, db=db0):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        elif type(arg) == str:
            super().__init__(db.blocks.find_one({"block.header.id.data": arg}))
        elif type(arg) == int:
            super().__init__(db.blocks.find_one({"block.header.height": arg}))

    @property
    def previousAssembly(self):
        return Assembly(self.previousAssemblyId.data, db=self.db)

    @property
    def transactions(self):
        result = []
        for t in self.db.transactions.find(
            {"blockId.data": self.block.header.id.data, "isCoinbase": False}
        ):
            result.append(Transaction(t))
        return result

    @property
    def coinbase(self):
        return Transaction(
            self.db.transactions.find_one(
                {"blockId.data": self.block.header.id.data, "isCoinbase": True}
            )
        )


class Assembly(CoolDict):
    def __init__(self, arg, db=db0):
        self.db = db
        if type(arg) == str:
            super().__init__(db.assemblies.find_one({"id.data": arg}))
        elif type(arg) == int:
            super().__init__(db.assemblies.find_one({"height": arg}))

    @property
    def previousAssembly(self):
        return Assembly(self.previousAssemblyId.data, db=self.db)


class Transaction(CoolDict):
    def __init__(self, arg, db=db0, pool=True):
        self.db = db
        if type(arg) == dict:
            super().__init__(arg)
        else:
            query = {"transaction.id.data": arg}
            if not pool:
                query["blockId"] = {"$exists": True}
            super().__init__(db.transactions.find_one(query))

    @property
    def block(self):
        return Block(self.blockId.data, db=self.db)


class CoolList(list):
    def __getattr__(self, name):
        return CoolList(getattr(x, name) for x in self)


class Transactions(CoolList):
    def __init__(self, arg=None, db=db0, pool=True):
        if type(arg) is str:
            query = {"transaction.id.data": arg}
            if not pool:
                query["blockId"] = {"$exists": True}
        elif type(arg) is dict:
            query = arg
        else:
            query = {}
        transactions = (Transaction(t) for t in db.transactions.find(query))
        super().__init__(transactions)

    @classmethod
    def by_input(cls, transaction_id, output_id, pool=True):
        query = {
            "transaction.inputs": {
                "$elemMatch": {
                    "id.data": transaction_id,
                    "outputId": output_id,
                }
            }
        }
        if not pool:
            query["blockId"] = {"$exists": True}
        return cls(query)


class Blocks:
    @classmethod
    def get_tip(cls, db=db0):
        return Block(db.blocks.find().sort("score", pymongo.DESCENDING).next())

In [6]:
Block(0, db=db1).previousAssembly.previousAssembly

{'_id': ObjectId('5cc954ea85699a69fd4e4f47'),
 'id': {'type': 'SHA256',
  'data': 'J0PpFDw5b9UVrYBUoc/q1NO0r1u9q4EXuAptGJYNf04='},
 'finishedComputation': True,
 'height': -2,
 'seed': 0,
 'nbAddresses': 2}

In [7]:
[
    (ob.get("branchPath"), ob["block"]["header"]["height"])
    for ob in db2.blocks.find({"branchPath.branchIds": {"$ne": [0]}})
]

[]

In [8]:
[
    (ob.get("branchPath"), ob["block"]["header"]["height"])
    for ob in db0.blocks.find({"branchPath.branchIds": {"$ne": [0]}})
]

[({'branchIds': [1, 0], 'blockNumbers': [0, 29]}, 379690),
 ({'branchIds': [2, 0], 'blockNumbers': [0, 29]}, 380219),
 ({'branchIds': [3, 0], 'blockNumbers': [0, 29]}, 382206),
 ({'branchIds': [4, 0], 'blockNumbers': [0, 29]}, 382551),
 ({'branchIds': [5, 0], 'blockNumbers': [0, 29]}, 382576),
 ({'branchIds': [6, 0], 'blockNumbers': [0, 30]}, 517676),
 ({'branchIds': [7, 0], 'blockNumbers': [0, 30]}, 517955)]

In [9]:
heights = [
    [ob["block"]["header"]["height"] for ob in db.blocks.find()]
    for db in (db0, db1, db2)
]

In [10]:
h0 = set(heights[0])
h1 = set(heights[1])
h2 = set(heights[2])

In [11]:
print(h0 - h1, h1 - h0)
print(h0 - h2, h2 - h0)
print(max(h0), len(h0))

{379690, 382576, 382578, 382551, 380219, 382206} set()
{517955, 379690, 517676, 382576, 382578, 382551, 380219, 382206} set()
517955 39


In [45]:
Transactions("1UM57qEdvRZ3V1TmMCjOetoH2URLglDPLSnAlEI6dxY=", pool=False).block.branchPath

[{'branchIds': [3, 0], 'blockNumbers': [0, 29]},
 {'branchIds': [5, 0], 'blockNumbers': [0, 29]},
 {'branchIds': [7, 0], 'blockNumbers': [0, 30]},
 {'branchIds': [4, 0], 'blockNumbers': [0, 29]},
 {'branchIds': [1, 0], 'blockNumbers': [0, 29]},
 {'branchIds': [0], 'blockNumbers': [31]},
 {'branchIds': [2, 0], 'blockNumbers': [0, 29]}]

In [65]:
Block("KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk=").branchPath

{'branchIds': [0], 'blockNumbers': [31]}

In [74]:
Transactions.by_input(transaction_id="jT0eSdxLjEpBGvHdy8WmgWzmXsKJm4Lq2xvKb7uwyu0=", output_id=1, pool=False).block.branchPath

[{'branchIds': [3, 0], 'blockNumbers': [0, 29]},
 {'branchIds': [5, 0], 'blockNumbers': [0, 29]},
 {'branchIds': [7, 0], 'blockNumbers': [0, 30]},
 {'branchIds': [4, 0], 'blockNumbers': [0, 29]},
 {'branchIds': [1, 0], 'blockNumbers': [0, 29]},
 {'branchIds': [0], 'blockNumbers': [31]},
 {'branchIds': [0], 'blockNumbers': [30]},
 {'branchIds': [2, 0], 'blockNumbers': [0, 29]}]

In [22]:
Block("KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk=")

{'_id': ObjectId('5cc9969e85699a12a228294d'),
 'block': {'header': {'id': {'type': 'SHA256',
    'data': 'KhlcPujzAs58+e9LT/2XeNueJEbZEvT4efAAtnLD6Uk='},
   'timestamp': {'data': 1556715166},
   'previousBlockHash': {'type': 'SHA256',
    'data': 'SWAo24vprvsuLKwkpuKHIlyfxB36QEPh0eUV1CnsGpA='},
   'author': {'signature': {'type': 'SHA256',
     'data': 'MQPzbioZt9p9ie0p43cq3/qMiVGaN54coHIPjqQnZ2nQHF/D5xlpQqn+B1F1cjhW6c1WYleHDDPQ7ChgsPB/9g=='},
    'keyPub': {'type': 'ECP256K1',
     'rawData': 'Awqh/GaK+zXAydO8h4sDf4apFNmW8SFaIqRqXioYDXy8'}},
   'height': 382578}},
 'branch': 'UNVERIFIED',
 'branchPath': {'branchIds': [0], 'blockNumbers': [31]}}